Quantum Convolutional Neural Network

This is a circuit that requires a connectivity map where the distance between the qubits increases exponentially with depth

In [1]:
from qiskit import QuantumCircuit

import time
import numpy as np

Define QCNN circuit

In [2]:
num_qubits = 10
num_layers = int(np.ceil(np.log2(num_qubits)))

qc=QuantumCircuit(num_qubits)
i_conv=0
for i_layer in range(num_layers):
    for i_sub_layer in [0 , 2**i_layer]:            
        for i_q1 in range(i_sub_layer, num_qubits, 2**(i_layer+1)):
            i_q2=2**i_layer+i_q1
            if i_q2<num_qubits:
                qc.rxx(np.random.rand(), i_q1, i_q2)
                qc.ry(np.random.rand(), i_q1)
                qc.ry(np.random.rand(), i_q2)
                i_conv+=1

Optionally print out the circuit

In [3]:
# qc.draw()

Define backend

In [4]:
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime.fake_provider import FakeTorino

# backend = GenericBackendV2(num_qubits=num_qubits, basis_gates = ['rz', 'rx', 'ry', 'cx'])
target = FakeTorino()

Compile with default passes

In [5]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=target, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0))

Time taken:  0.5806818008422852
OrderedDict({'sx': 135, 'rz': 127, 'cz': 52, 'x': 15})
Number of 2-qubit gates:  52
Number of 1-qubit gates:  127


In [6]:
# print(default_qc)

UCC

In [11]:
#Test with UCC transpiler
import sys
sys.path.append('../')

from ucc import compile

t1 = time.time()
ucc_qc, gate_counts = compile(qc, mode="ucc", get_gate_counts = True, target=target)
t2 = time.time()
print("Time taken: ", t2-t1)

# # Get gate counts
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

Time taken:  0.18474245071411133
OrderedDict({'cx': 53, 'ry': 42, 'rz': 36})
Number of 2-qubit gates:  53
Number of 1-qubit gates:  78


In [8]:
# ucc_qc.draw()

Create custom compiler

In [9]:
# from qiskit.transpiler import PassManager
# import qiskit.transpiler.passes as passes 

# custom_pm = PassManager()

# custom_pm.append(passes.Collect2qBlocks())
# custom_pm.append(passes.ConsolidateBlocks())

# # custom_pm.append(passes.Decompose())
# # custom_pm.append(passes.Optimize1qGatesDecomposition())

# custom_qc = custom_pm.run(qc)

# gate_counts = custom_qc.count_ops()
# print(gate_counts)

# custom_qc.draw()

In [10]:
# # Get gate counts
# gate_counts = custom_qc.count_ops()
# print(gate_counts)
# print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
# print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))
